<a href="https://colab.research.google.com/github/hacksasatw-cmd/Torrent_To_Google_Drive_Downloader/blob/main/Torrent_To_Google_Drive_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torrent To Google Drive Downloader

**Important Note:** To get more disk space:
> Go to Runtime -> Change Runtime and give GPU as the Hardware Accelerator.  You will get around 384GB to download any torrent you want.

### Install libtorrent and Initialize Session

In [1]:
# Uninstall potentially incompatible packages
!apt-get remove python3-libtorrent --yes
!pip uninstall lbry-libtorrent --yes

# Install the correct, compatible `libtorrent` wheel
!pip install libtorrent

# Import and initialize libtorrent
import libtorrent as lt
ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []
print("libtorrent version:", lt.version)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package 'python3-libtorrent' is not installed, so not removed
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 54.8 MB/s eta 0:00:00
libtorrent version: 2.0.11.0


/tmp/ipython-input-1640062018.py:11: DeprecationWarning: listen_on() is deprecated
  ses.listen_on(6881, 6891)


### Mount Google drive and Add From Torrent File
You can run this cell to add more files as many times as you want

In [5]:
from google.colab import files

from google.colab import drive

drive.mount("/content/drive")
print("Google drive mounted")

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google drive mounted


KeyboardInterrupt: 

### Add From Magnet Link
You can run this cell to add more files as many times as you want

In [ ]:
params = {"save_path": "/content/drive/My Drive/Torrent"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


### Start Download
Source: https://stackoverflow.com/a/5494823/7957705 and [#3 issue](https://github.com/FKLC/Torrent-To-Google-Drive-Downloader/issues/3) which refers to this [stackoverflow question](https://stackoverflow.com/a/6053350/7957705)

In [3]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "completed")
    time.sleep(1)


FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

/tmp/ipython-input-3675360783.py:30: DeprecationWarning: is_seed() is deprecated
  if not download.is_seed():
/tmp/ipython-input-3675360783.py:35: DeprecationWarning: name() is deprecated
  download.name(),
/tmp/ipython-input-3675360783.py:48: DeprecationWarning: name() is deprecated
  print(download.name(), "completed")


www.1TamilMV.mba - Madharasi (2025) Tamil HQ HDRip - 1080p - HEVC - x265 - (DD+5.1 - 192Kbps & AAC) - 1.5GB - ESub.mkv completed
